In [ ]:
!pip install git+https://github.com/rcmalli/keras-squeezenet

import tensorflow as tf
from keras_squeezenet import SqueezeNet

from google.colab import drive
drive.mount('/content/drive')

conv_base = SqueezeNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

conv_base.summary()

#  dondurma için
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'fire9/expand3x3':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model = tf.keras.models.Sequential()

model.add(conv_base)

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=['acc'])

model.summary()



In [ ]:
# Veri seti 1 yolu
train_dir = '/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/train'
validation_dir = '/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/Validation'
test_dir = '/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/test'

In [ ]:
# Veri seti hard yolu
train_dir = '/content/drive/My Drive/Image_Processing/Veri_Setleri//veri_seti_hard/Train'
validation_dir = '/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_hard/Validation'
test_dir = '/content/drive/My Drive/Image_Processing/Veri_Setleri//veri_seti_hard/Test'

In [ ]:

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=20,
)

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=5
)

model.save('trained_tf_model.h5')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=20,
)

test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('test acc:', test_acc)
